# Experiment Template

Short description of the goal, dataset, and metric.

- Dataset: CIFAR-10
- Model: small CNN / ResNet18
- Metric: Top-1 accuracy


In [ ]:
# Optional: mount Google Drive to persist large files (models, datasets)
from google.colab import drive
drive.mount('/content/drive')
EXP_ROOT = '/content/drive/MyDrive/cv-lab/experiments'  # change if needed

import os, datetime
exp_name = datetime.datetime.now().strftime('%Y-%m-%d') + '_cifar10_baseline'
exp_dir = os.path.join(EXP_ROOT, exp_name)
os.makedirs(exp_dir, exist_ok=True)
print('Experiment dir:', exp_dir)


In [ ]:
# Setup: PyTorch + CIFAR-10 (from tutorial)
import torch, torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
trainset = datasets.CIFAR10(root='/content/data', train=True, download=True, transform=transform)
testset  = datasets.CIFAR10(root='/content/data', train=False, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader  = DataLoader(testset, batch_size=256, shuffle=False, num_workers=2)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


In [ ]:
# Simple model (e.g., ResNet18 from torchvision)
model = torchvision.models.resnet18(num_classes=10)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)


In [ ]:
# Training loop (short)
for epoch in range(2):
    model.train()
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if (i+1) % 50 == 0:
            print(f"[Epoch {epoch+1}] Step {i+1} Loss {running_loss/50:.4f}")
            running_loss = 0.0
print('Done!')


In [ ]:
# Save small artifacts (plots, metrics) to exp_dir
import json
metrics = {"note": "Fill with accuracy etc."}
with open(os.path.join(exp_dir, 'metrics.json'), 'w') as f:
    json.dump(metrics, f, indent=2)

# Save model to Drive (NOT to Git)
torch.save(model.state_dict(), os.path.join(exp_dir, 'model.pth'))
print('Saved metrics + model to', exp_dir)
